In [1]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import preprocessor
import display
import classifier
import csv_file_io
import datetime


PROJECT_HOME = './'
# DRONE = 'Inspire2' # 'Bebop2', 'Spark', 'MavicPro', 'Phantom4', 'Matrice100', 'MavicAir', 'AutelEVO', 'Inspire2', 'JME', 'ParrotAnafi']
# DATASET_DIR = PROJECT_HOME + 'dataset/validation/Inspire2/Inspire2_validation_File3.wav'
RESULT_FILE_DIR = PROJECT_HOME + 'results/'
DATASET_FILE_DIR = PROJECT_HOME + 'results/dataset/'
TRAINING_DATASET_DIR = PROJECT_HOME + 'dataset/1. training/'
VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/2. validation/'
TEST_DATASET_DIR = PROJECT_HOME + 'dataset/3. test/'
TRAINING_VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/4. training_validation/'
TRAINING_VALIDATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/5. training_validation_test/'
SEGMENTATION_DATASET_DIR = PROJECT_HOME + 'dataset/6. segmentation/'
SEGMENTATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/7. segmentation(test)/'

# now = datetime.datetime.now()
# now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

RESULT_FILE_NAME = "[Cross-Validation] result_"

SAMPLE_RATE = 44100
FIG_SIZE = (15,5)

FRAME_LENGTH = 0.250 # 250 ms
FRAME_STRIDE = 0.250 # same with frame_length for 0% overlap

DIGITS = 3

b_visualizing = False

def save_cv_classification_performance(kernel: str, report_dict: dict, acc: float, date_time: datetime, standardscaler: bool):
    if standardscaler == True:
        name = '_scale(O)_'
    else:
        name = '_scale(X)_'

    file_name = RESULT_FILE_NAME.split('_')[0] + name
    f = csv_file_io.open_csv(RESULT_FILE_DIR + file_name + date_time + '.csv', 'a')
    csv_file_io.write_csv_data(f, [kernel + "'s Cross-validation report"])
    csv_file_io.write_csv_data(f, ['', 'fit_time', 'score_time', 'test_score'])
    
    for i in range(0, len(report_dict['test_score'])):
        csv_file_io.write_csv_data(f, [i, report_dict['fit_time'][i], report_dict['score_time'][i],  report_dict['test_score'][i]])    
        
    csv_file_io.write_csv_data(f, ['', '', 'accuracy', acc])    
    csv_file_io.write_csv_data(f, [])

    f.close()

c:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


importing Jupyter notebook from preprocessor.ipynb
importing Jupyter notebook from display.ipynb
importing Jupyter notebook from classifier.ipynb
importing Jupyter notebook from csv_file_io.ipynb


In [2]:
dict_integrated_train_validation_test_signal = preprocessor.get_integrated_signal(SEGMENTATION_DATASET_DIR, type='SEGMENTATION')

# Visualizing Audio (waveform and power spectrum)
if b_visualizing == True:
    for key, val in dict_integrated_train_validation_test_signal.items():
        display.display_waveshow(val, key + "'s Training+Validation+Test Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Training+Validation+Test Signal's Power Spectrum")

SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (44100,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (88200,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (132300,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (176400,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (220500,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (264600,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (308700,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (352800,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated signal :  (396900,)
SEGMENTATION / AutelEVO's signal :  (44100,)
SEGMENTATION / AutelEVO's integrated si

In [3]:
# Feature Extraction (MFCC)
N_MFCC = 40
n_fft = int(SAMPLE_RATE * FRAME_LENGTH) # length of the FFT window, frame 하나당 sample의 수 / n_fft=2048, n_fft = int(SAMPLE_RATE * FRAME_LENGTH)
hop_length = int(SAMPLE_RATE * FRAME_STRIDE) # number of samples between successive frames / hop_length=512, hop_length = int(SAMPLE_RATE * FRAME_STRIDE)
n_mels = 128
f_max = 8000

dict_melspectrogram_train_validation_test_signal = {}
for key, val in dict_integrated_train_validation_test_signal.items():    
    dict_melspectrogram_train_validation_test_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)    

dict_mfccs_train_validation_test_signal = {}
for key, val in dict_melspectrogram_train_validation_test_signal.items():
    dict_mfccs_train_validation_test_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

# print(dict_melspectrogram_train_validation_test_signal['AutelEVO'].shape)
# print(dict_mfccs_train_validation_test_signal['AutelEVO'].shape)

if b_visualizing == True:
    for key, val in dict_mfccs_train_validation_test_signal.items():
        display.display_mfccs(val,  "'s Training+Validation+Test Signal's MFCCs")  

In [4]:
arr_train_validation_test_data = None
arr_train_validation_test_target = None

for key, val in dict_mfccs_train_validation_test_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_train_validation_test_data is None:
        arr_train_validation_test_data = arr_mfccs_data
        arr_train_validation_test_target = arr_target        
    else:       
        arr_train_validation_test_data = np.vstack((arr_train_validation_test_data, arr_mfccs_data))
        arr_train_validation_test_target = np.vstack((arr_train_validation_test_target, arr_target))

print("Training+Validation+Test data / target : ", arr_train_validation_test_data.shape, arr_train_validation_test_target.shape)
# ararr_train_validation_test_data_target = np.hstack((arr_train_validation_test_data, arr_train_validation_test_target))
# np.savetxt(DATASET_FILE_DIR + DATASET_FILE_NAME + now_date_time + '.csv', ararr_train_validation_test_data_target, delimiter=",", fmt='%s')

Training+Validation+Test data / target :  (14933, 40) (14933, 1)


In [6]:
now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

CV = 10
b_standardscaler = True

report, acc = classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("linear", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("LinearSVC", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("rbf", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("poly", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("sigmoid", report, acc, now_date_time, b_standardscaler)


kernel(linear)'s Cross-validation report:
    fit_time  score_time  test_score
0  4.930371    0.393947    0.518072
1  4.793244    0.381957    0.389558
2  4.859604    0.382975    0.469210
3  4.825093    0.380979    0.526457
4  4.886928    0.391952    0.559277
5  4.848032    0.381978    0.484930
6  4.863423    0.387962    0.466845
7  4.799011    0.378986    0.482251
8  4.790884    0.377496    0.493637
9  4.836065    0.387962    0.484260
kernel(linear)'s Cross-validation Accuracy: 0.557

LinearSVC's Cross-validation report:
      fit_time  score_time  test_score
0  122.022410    0.000998    0.477242
1  113.956521    0.000998    0.472557
2  116.124854    0.000998    0.416332
3  117.437279    0.000997    0.569324
4  118.016380    0.000998    0.424648
5  116.544877    0.000998    0.618888
6  124.229803    0.000998    0.480241
7  116.555013    0.000998    0.441393
8  116.043752    0.000998    0.508372
9  118.710730    0.000997    0.540522
LinearSVC's Cross-validation Accuracy: 0.64

kernel(r

In [5]:
# arr_std_train_validation_test_data = classifier.set_scaler_train(arr_train_validation_test_data)

now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

CV = 10
b_standardscaler = False

report, acc = classifier.cross_validation_svm("linear", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("linear", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("LinearSVC", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("LinearSVC", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("rbf", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("rbf", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("poly", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("poly", report, acc, now_date_time, b_standardscaler)

report, acc = classifier.cross_validation_svm("sigmoid", CV, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_cv_classification_performance("sigmoid", report, acc, now_date_time, b_standardscaler)



kernel(linear)'s Cross-validation report:
    fit_time  score_time  test_score
0  5.481855    0.466750    0.518072
1  5.222537    0.413891    0.389558
2  5.254506    0.399929    0.469210
3  5.459954    0.403918    0.526457
4  5.390652    0.446805    0.559277
5  5.551684    0.404917    0.484930
6  5.473375    0.442814    0.466845
7  5.611003    0.415890    0.482251
8  5.332751    0.435835    0.493637
9  5.581129    0.416755    0.484260
kernel(linear)'s Cross-validation Accuracy: 0.487

LinearSVC's Cross-validation report:
      fit_time  score_time  test_score
0  138.782904    0.000997    0.741633
1  123.912982    0.000998    0.542838
2  128.235745    0.000998    0.504685
3  128.232517    0.000997    0.527127
4  135.615246    0.000997    0.498326
5  129.352173    0.001995    0.496316
6  135.117613    0.000998    0.476892
7  129.612848    0.000996    0.415271
8  133.614913    0.001995    0.466175
9  134.154067    0.000997    0.497656
LinearSVC's Cross-validation Accuracy: 0.502

kernel(